In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Feature extraction

In [2]:
import glob
import os
import numpy as np

from PIL import Image, ImageFile
from omegaconf import OmegaConf

from config.init import create_baseconfig_from_checkpoint
from model.lgffem import LGFFEM

from utils.revisitop.dataset import configdataset
from utils.revisitop.evaluate import compute_map

from tqdm import tqdm

In [3]:
import torch
from torchvision.transforms import v2
from torch.utils.data import Dataset

from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create model from checkpoint

In [4]:
path_checkpoint = os.path.join('/thesis/checkpoint/20240212_2301-EMB-11_convnextv2-02_neck_512_3-00_head_A-epoch4.pth')
checkpoint = torch.load(path_checkpoint)

In [5]:
base_config = create_baseconfig_from_checkpoint(checkpoint)

embedder = LGFFEM(base_config).eval().to(device)
match_n = embedder.neck.load_state_dict(checkpoint['model_neck_state_dict'], strict = False)
print('[++] Loaded neck weights.', match_n)
match_h = embedder.head.load_state_dict(checkpoint['model_head_state_dict'], strict = False)
print('[++] Loaded head weights.', match_h)

[++] Loaded neck weights. <All keys matched successfully>
[++] Loaded head weights. <All keys matched successfully>


In [6]:
img_transforms = v2.Compose([
                            v2.ToImage(),
                            v2.Resize(size=(224, 224)),
                            v2.ToDtype(torch.float32, scale=True),
                            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                          ])

## Extraction features dataset: `oxford5k`

In [7]:
data_root = os.path.join('/thesis/classical/revisitop')

In [8]:
# Set test dataset: roxford5k | rparis6k
test_dataset = 'roxford5k'

def pil_loader(path):
    # to avoid crashing for truncated (corrupted images)
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    # open path as file to avoid ResourceWarning 
    # (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

print('>> {}: Processing test dataset...'.format(test_dataset)) 
# config file for the dataset
# separates query image list from database image list, if revisited protocol used
cfg = configdataset(test_dataset, os.path.join(data_root, 'datasets'))

>> roxford5k: Processing test dataset...


In [9]:
# query images
qim_embs_l = []
for i in tqdm(np.arange(cfg['nq'])):
    qim = pil_loader(cfg['qim_fname'](cfg, i)).crop(cfg['gnd'][i]['bbx'])
    qim = img_transforms(qim).unsqueeze(0).to(device)
    qim_embs_l.append(embedder(qim).detach().cpu().numpy())

Q = np.swapaxes(np.stack(qim_embs_l, axis=0).squeeze(1),0,1)

100%|██████████| 70/70 [00:02<00:00, 29.03it/s]


In [10]:
im_embs_l = []
for i in tqdm(np.arange(cfg['n'])):
    im = pil_loader(cfg['im_fname'](cfg, i))
    im = img_transforms(im).unsqueeze(0).to(device)
    im_embs_l.append(embedder(im).detach().cpu().numpy())

X = np.swapaxes(np.stack(im_embs_l, axis=0).squeeze(1),0,1)

100%|██████████| 4993/4993 [02:31<00:00, 32.92it/s]


### Retrieval Option 1

In [11]:
# perform search
print('>> {}: Retrieval...'.format(test_dataset))
sim = np.dot(X.T, Q)
ranks = np.argsort(-sim, axis=0)

>> roxford5k: Retrieval...


In [12]:
# revisited evaluation
gnd = cfg['gnd']

# evaluate ranks
ks = [1, 5, 10]

In [13]:
# search for easy
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
    gnd_t.append(g)
mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, ks)

In [14]:
# search for easy & hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk']])
    gnd_t.append(g)
mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, ks)

In [15]:
# search for hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    gnd_t.append(g)
mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, ks)

In [16]:
print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))

>> roxford5k: mAP E: 19.18, M: 14.28, H: 4.11
>> roxford5k: mP@k[ 1  5 10] E: [25.   23.68 21.03], M: [27.14 25.14 20.57], H: [10.   10.86  7.57]


### Retrieval Option 2

In [17]:
import faiss

In [18]:
dim, size = X.shape
m_neighbors = 12

In [19]:
# quantizer = faiss.IndexFlatIP(dim)
# index = faiss.IndexIVFPQ(quantizer, dim, 316, 128, 8)

# index = faiss.IndexHNSWFlat(dim, m_neighbors)
index = faiss.IndexFlatL2(dim) ##euclidean
# index = faiss.IndexFlatIP(dim) ##cosine

In [20]:
index.train(np.swapaxes(X,0,1))
index.add(np.swapaxes(X,0,1))

In [21]:
sims, ranks_f = index.search(np.swapaxes(Q,0,1), size)
ranks_f = np.swapaxes(ranks_f,0,1)

In [22]:
# revisited evaluation
gnd = cfg['gnd']

# evaluate ranks
ks = [1, 5, 10]

In [23]:
# search for easy
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
    gnd_t.append(g)
mapE, apsE, mprE, prsE = compute_map(ranks_f, gnd_t, ks)

In [24]:
# search for easy & hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk']])
    gnd_t.append(g)
mapM, apsM, mprM, prsM = compute_map(ranks_f, gnd_t, ks)

In [25]:
# search for hard
gnd_t = []
for i in range(len(gnd)):
    g = {}
    g['ok'] = np.concatenate([gnd[i]['hard']])
    g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
    gnd_t.append(g)
mapH, apsH, mprH, prsH = compute_map(ranks_f, gnd_t, ks)

In [26]:
print('>> {}: mAP E: {}, M: {}, H: {}'.format(test_dataset, np.around(mapE*100, decimals=2), np.around(mapM*100, decimals=2), np.around(mapH*100, decimals=2)))
print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(test_dataset, np.array(ks), np.around(mprE*100, decimals=2), np.around(mprM*100, decimals=2), np.around(mprH*100, decimals=2)))

>> roxford5k: mAP E: 20.36, M: 15.11, H: 3.86
>> roxford5k: mP@k[ 1  5 10] E: [25.   26.99 24.65], M: [25.71 27.43 23.71], H: [8.57 7.21 6.36]


In [ ]:
thesis/classical/test-patches-kimia/s0